## Association Rule Mining in Retail Store

### Problem Statement:
 * What are the items that may be frequently purchased together?

### Objective:
* To learn how Apriori Algorithm and Association Rules works.
* To learn how Combination and Permutation helps to find Support and Confidence of itemsets respectively.
* To find frequent itemsets with high confidence and lift, keeping both item together will help to increase sales.


### Introduction
* Association rule mining is one of an important technique of data mining for knowledge discovery.
* The knowledge of the correlation between the items in the data transaction can use association rule mining.
* Retail store analysis is one of an application area of association rule mining technique.
* The possible percentage of the correlation of combined items gives the new knowledge. Therefore, it is a very helpful for determiner to take the decisions

### Analysis

In [1]:
# Importing Required Library

import pandas as pd
import numpy as np

In [2]:
# Reading Excel file 
# need to install 1.2.0 version xlrd package to read excel files: pip install xlrd==1.2.0
bread = pd.read_excel('raw_bread.xlsx')

In [3]:
## Here we have transaction data, which include column, Date,Time,Transaction,Item
## we should remove duplicate transaction, it shows quantity of item in same transaction,
## it is not needed in appriori aglo as we only care about different item in particular transaction
bread

,"Date,Time,Transaction,Item"
0,"2016-10-30,09:58:11,1,Bread"
1,"2016-10-30,10:05:34,2,Scandinavian"
2,"2016-10-30,10:05:34,2,Scandinavian"
3,"2016-10-30,10:07:57,3,Hot chocolate"
4,"2016-10-30,10:07:57,3,Jam"
5,"2016-10-30,10:07:57,3,Cookies"
6,"2016-10-30,10:08:41,4,Muffin"
7,"2016-10-30,10:13:03,5,Coffee"
8,"2016-10-30,10:13:03,5,Pastry"
9,"2016-10-30,10:13:03,5,Bread"


In [4]:
## dropping Duplicate Transaction
bread = bread.drop_duplicates()

In [5]:
## we need to split transaction data into Dataframe/tabular structure as follow
new = bread['Date,Time,Transaction,Item'].str.split(',', n = 3, expand = True)

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
## assigning column to data frame "bread"
bread['Date'] = new[0]
bread['Time'] = new[1]
bread['Transaction'] = new[2]
bread['Item'] = new[3]

In [8]:
# in this dataframe we only need column Trasaction and Item, rest is not needed in association mining rule
bread[['Date', 'Time', 'Transaction', 'Item']].head(10)
                                                    

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies
6,2016-10-30,10:08:41,4,Muffin
7,2016-10-30,10:13:03,5,Coffee
8,2016-10-30,10:13:03,5,Pastry
9,2016-10-30,10:13:03,5,Bread
10,2016-10-30,10:16:55,6,Medialuna


In [9]:
# we need to convert cloumn transacton & item into Crosstab or we can say Binary Matrix as follow
transaction = pd.crosstab(index= bread['Transaction'], columns= bread['Item'])

In [10]:
## Just writing csv file to check result
## we have one unwanted column named "NONE", we should remove it as follow and proceed further
#tab.to_csv('tab.csv')

In [11]:
## removing unwanted col "NONE"
transaction = transaction.drop(['NONE'], axis = 1)

### Creating APRIORI function to generate frequent itesets based on minimum threshold support = 0.04

In [12]:
def APRIORI(data, min_support = 0.04,  max_length = 4):
    # Collecting Required Library
    import numpy as np
    import pandas as pd
    from itertools import combinations
    
    support = pd.DataFrame
    L = list(data.columns)
    
    #第一次
    L = list(transaction.columns)
    n = []
    a=list(combinations(L, 1))
    for t in a:
        n.append(float(transaction[list(t)].mean().values))
    n=pd.DataFrame(list(zip(a,n)))
    n.columns=["Item","Support"]
    L1=n[n["Support"]>=0.04]
    yes_item=[x[0] for x in L1["Item"]]
    support=L1
    #第二次以後 
    new_L=[]
    for number in range(2,max_length+1):
        last_L=new_L
        a=list(combinations(yes_item, number))
        c=[]
        if number==2:
            c=a
        else:
            for h in a:
                if(all(i in last_L["Item"] for i in list(combinations(h, number-1)))):
                    c.append(h)
        n = []
        for h in c:
            b=transaction[list(h)]
            length=len(h)
            prob=(b.apply(sum, axis=1)==length).mean()
            n.append(prob)
        if (n!=[]):
            n=pd.DataFrame(list(zip(c,n)))
            n.columns=["Item","Support"]
            new_L=n[n["Support"]>=0.04]
            support=pd.concat([support,new_L])
    result = support
    return result

In [13]:
## finding frequent itemset with min support = 4%
my_freq_itemset = APRIORI(transaction, 0.04, 3)
my_freq_itemset.sort_values(by = ['Support'], ascending = False)

,Item,Support
23,"(Coffee,)",0.475081
11,"(Bread,)",0.324940
83,"(Tea,)",0.141643
15,"(Cake,)",0.103137
1,"(Bread, Coffee)",0.089393
65,"(Pastry,)",0.085510
73,"(Sandwich,)",0.071346
55,"(Medialuna,)",0.061379
48,"(Hot chocolate,)",0.057916
8,"(Cake, Coffee)",0.054349


### Creating ASSOCIATION_RULE function to generate rules based on minimun threshold confidence.

In [14]:
def ASSOCIATION_RULE(df, min_confidence_threshold = 0.5):
    import pandas as pd
    from itertools import combinations
    
    df.index=my_freq_itemset.Item
    support = pd.Series(df.Support, index=df.Item).to_dict()
    antecedents=[]
    consequents=[]
    antecedent_support=[]
    consequent_support=[]
    supports=[]
    confidence=[]
    lift=[]
    leverage=[]
    conviction=[]
    for x in my_freq_itemset.Item:
        if len(x)>1:
            assosiation=list(combinations(x, len(x)-1))
            for y in assosiation:
                repuca=assosiation[:]
                repuca.remove(y)
                consequents.append(repuca[0])
                antecedents.append(y)
                consequent_support.append(round(support[repuca[0]],6))
                antecedent_support.append(round(support[y],6))
                supports.append(round(support[x],6))
                confidence.append(round(support[x]/support[y],6))
                lift.append(round(support[x]/support[y]/support[repuca[0]],6))
                leverage.append(round(support[x]-support[y]*support[repuca[0]],6))
                conviction.append(round((1-support[repuca[0]])/(1-support[x]/support[y]),6))
    data=pd.DataFrame([antecedents,consequents,antecedent_support,consequent_support,supports,confidence,lift,leverage,conviction]).T
    data.columns=["antecedents", "consequents", "antecedent support", "consequent support","support", "confidence", "Lift", "Leverage", "Convection"]
    data=data[data["confidence"]>=min_confidence_threshold]
    result = data
    return result

In [15]:
## Rule with minimun confidence = 50%
my_rule = ASSOCIATION_RULE(my_freq_itemset, 0.5)
my_rule
# the column names in the following table is the same as those generated by mlxtend
# you can refer to http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/ 
# to find the formulas of lift, leverage, convection

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
2,"(Cake,)","(Coffee,)",0.103137,0.475081,0.054349,0.526958,1.1092,0.00535,1.10967
5,"(Pastry,)","(Coffee,)",0.08551,0.475081,0.047214,0.552147,1.16222,0.00659,1.17208


### Finally sorting results by Lift to get highly associated itemsets.

In [16]:
my_rule.sort_values(by='Lift', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
5,"(Pastry,)","(Coffee,)",0.08551,0.475081,0.047214,0.552147,1.16222,0.00659,1.17208
2,"(Cake,)","(Coffee,)",0.103137,0.475081,0.054349,0.526958,1.1092,0.00535,1.10967


## Cross Verifying results with  apriori and association rule from mlxtend

In [17]:
# Loading standard package
from mlxtend.frequent_patterns import apriori, association_rules

In [18]:
## finding frequent itemset with min support = 4%
frequent_itemset = apriori(df = transaction, min_support= 0.04, use_colnames= True)
frequent_itemset.sort_values(by = 'support', ascending = False)

,support,itemsets
2,0.475081,(Coffee)
0,0.324940,(Bread)
8,0.141643,(Tea)
1,0.103137,(Cake)
9,0.089393,"(Bread, Coffee)"
6,0.085510,(Pastry)
7,0.071346,(Sandwich)
5,0.061379,(Medialuna)
4,0.057916,(Hot chocolate)
10,0.054349,"(Cake, Coffee)"


### Createing associate rule such that item brought with conditional probability(Confidence) more than 50% with corresponding item

In [19]:
## Rule with minimun confidence = 50%
Rules = association_rules(frequent_itemset, min_threshold= 0.5)
Rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667
1,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079


In [20]:
## Finally sorting results by Lift to get highly associated itemsets.
Rules.sort_values(by='lift', ascending= False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079
0,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667


### Conclusion
 * Results from developed function(APRIORI, ASSOCIATION_RULE) has matched with builts packages.
 * it is observed that "Toast" & "Coffee" are highly associated with lift 1.48.
 * Coffee has been brought most frequently with 47.5% of all the transactions